### wps_climdex_rxnday

WPS wrapper for [climdex.pcic](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf)'s rx1day and rx5day functions

**Monthly Maximum 1-day Precipitation** ([climdex.rx1day](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=21)) This function takes a climdexInput object as input and computes the climdex index Rx1day: monthlyor annual maximum 1-day precipitation.

**Monthly Maximum 5-day Consecutive Precipitation** ([climdex.rx5day](https://cran.r-project.org/web/packages/climdex.pcic/climdex.pcic.pdf#page=23)) This function takes a climdexInput object as input and computes the climdex index Rx5day: monthlyor annual maximum 5-day consecutive precipitation.

In [1]:
import os
import requests
from birdy import WPSClient
from rpy2 import robjects
from urllib.request import urlretrieve
from importlib.resources import files
from tempfile import NamedTemporaryFile

from wps_tools.R import rda_to_vector, construct_r_out, test_rda_output
from wps_tools.testing import get_target_url

In [2]:
# Ensure we are in the working directory with access to the data
while os.path.basename(os.getcwd()) != "quail":
    os.chdir('../')

In [3]:
# NBVAL_IGNORE_OUTPUT
url = get_target_url("quail")
print(f"Using quail on {url}")

Using quail on https://marble-dev01.pcic.uvic.ca/twitcher/ows/proxy/quail/wps


In [4]:
quail = WPSClient(url)

#### Help for individual processes can be diplayed using the ? command (ex/ bird.process?)

In [5]:
# NBVAL_IGNORE_OUTPUT
quail.climdex_rxnday?

Signature:
quail.climdex_rxnday(
    climdex_input,
    num_days=None,
    loglevel='INFO',
    output_file='output.rda',
    freq='monthly',
    center_mean_on_last_day=False,
    output_formats=None,
)
Docstring:
Computes the mean daily diurnal temperature range.

Parameters
----------
climdex_input : ComplexData:mimetype:`application/x-gzip`
    RDS or Rdata (.rds, .rda, .rdata) file containing R Object of type climdexInput
output_file : string
    Filename to store the output Rdata (extension .rda)
freq : {'monthly', 'annual'}string
    Time frequency to aggregate to
num_days : {'1', '5'}positiveInteger
    Compute rx[1]day or rx[5]day
center_mean_on_last_day : boolean
    Whether to center the 5-day running mean on the last day of the window, insteadof the center day.
loglevel : {'CRITICAL', 'ERROR', 'WARNING', 'INFO', 'DEBUG', 'NOTSET'}string
    Logging level

Returns
-------
rda_output : ComplexData:mimetype:`application/x-gzip`
    Rda file containing R output data
File:      

#### Run wps_climdex_rxnday Process with rx1day with rda input

In [6]:
with NamedTemporaryFile(suffix=".rda", prefix="rx1day_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_rxnday(
            climdex_input=(files("tests") / "data/climdexInput.rda").resolve(),
            freq="annual",
            num_days=1,
            output_file=output_file.name,
        )
rx1day_url = output.get()[0]

#### Run wps_climdex_rxnday Process with rx5day with rds input

In [7]:
with NamedTemporaryFile(suffix=".rda", prefix="rx5day_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_rxnday(
            climdex_input=(files("tests") / "data/climdexInput.rds").resolve(),
            freq="annual",
            num_days=5,
            center_mean_on_last_day=False,
            output_file=output_file.name,
        )
rx5day_url = output.get()[0]

You can also have multiple inputs 

In [8]:
climdex_inputs = [
    (files("tests") / "data/climdexInput.rds").resolve(),
    (files("tests") / "data/climdexInput.rda").resolve(),
    (files("tests") / "data/climdex_input_multiple.rda").resolve(),
]
with NamedTemporaryFile(suffix=".rda", prefix="rx5day_", dir="/tmp", delete=True) as output_file:
    output = quail.climdex_rxnday(
            climdex_input=climdex_inputs,
            freq="annual",
            num_days=5,
            center_mean_on_last_day=False,
            output_file=output_file.name,
        )
rx5day_url = output.get()[0]

Access the output with **rda_to_vector** or **construct_r_out** from **wps_tools.R**

In [12]:
# NBVAL_IGNORE_OUTPUT
rx1day = rda_to_vector(rx1day_url, "rx1day1_ci")
# use print() to see whole vector
print(f"rx1day\n{rx1day}")
rx5day = rda_to_vector(rx5day_url, "rx5day1_ci")
print(f"rx5day\n{rx5day}")

rx1day
1959 1960 1961 1962 1963 1964 1965 1966 1967 1968 1969 1970 1971 1972 1973 1974 
  NA 50.8   NA   NA   NA 38.1 34.8 30.2 45.2 82.3 24.9 27.9 66.3 64.8 42.7 57.9 
1975 1976 1977 1978 1979 1980 1981 1982 1983 1984 1985 1986 1987 1988 1989 1990 
56.9   NA   NA 37.2 50.0 62.0 32.2   NA 54.2   NA   NA   NA   NA   NA 58.2 91.6 
1991 1992 1993 1994 1995 1996 1997 1998 1999 2000 2001 2002 2003 2004 
  NA 52.8   NA   NA 88.0   NA 47.6 33.5 41.8 33.0 36.4   NA 76.0   NA 

rx5day
 1959  1960  1961  1962  1963  1964  1965  1966  1967  1968  1969  1970  1971 
   NA  66.1    NA    NA    NA  77.0  78.4  83.1  97.6  94.7  73.2  64.7 102.1 
 1972  1973  1974  1975  1976  1977  1978  1979  1980  1981  1982  1983  1984 
134.9  60.0 113.1 162.8    NA    NA  74.6 142.4  91.4  87.2    NA 129.6    NA 
 1985  1986  1987  1988  1989  1990  1991  1992  1993  1994  1995  1996  1997 
   NA    NA    NA    NA 141.5 170.2    NA  87.1    NA    NA 151.0    NA  84.0 
 1998  1999  2000  2001  2002  2003  2004 
12

In [10]:
# NBVAL_IGNORE_OUTPUT
construct_r_out([rx1day_url, rx5day_url])

[[R object with classes: ('numeric',) mapped to:
  [     nan, 50.799999,      nan,      nan, ..., 36.400002,      nan, 76.000000,      nan]],
 [R object with classes: ('numeric',) mapped to:
  [     nan, 66.099999,      nan,      nan, ..., 99.600003,      nan, 164.400000,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 66.099999,      nan,      nan, ..., 99.600003,      nan, 164.400000,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 66.099999,      nan,      nan, ..., 99.600003,      nan, 164.400000,      nan],
  R object with classes: ('numeric',) mapped to:
  [     nan, 66.099999,      nan,      nan, ..., 99.600003,      nan, 164.400000,      nan]]]

#### Test output against expected output

In [11]:
test_rda_output(
    rx1day_url, "rx1day1_ci", "expected_rxnday.rda", "expected_rx1day_annual"
)
test_rda_output(
    rx5day_url, "rx5day1_ci", "expected_rxnday.rda", "expected_rx5day_annual"
)